In [1]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras

import urllib.request
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow.keras import backend as K

In [3]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

2021-12-06 10:04:30.886522: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-12-06 10:04:30.902441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 10:04:30.903342: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-SXM2-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.53
pciBusID: 0000:00:05.0
2021-12-06 10:04:30.903490: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-12-06 10:04:30.903561: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcubla

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Lambda, Input

In [5]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]

In [6]:
lblist = ['범죄','혐오','선정','비도덕 아님','폭력','비난','욕설','차별']
e_lblist = ['crime', 'hate', 'sexual', 'none', 'violent','blame', 'curse','discrimination']

In [7]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
for i,lb in enumerate(e_lblist):
    data = pd.read_csv('./data_'+lb+'.csv')
    train,test =train_test_split(data,train_size = 0.9)
    x_train = train['sentence']
    x_test = test['sentence']
    y_train = train[lblist[i]]
    y_test = test[lblist[i]]

    model = Model(inputs=[input_text], outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=10, batch_size=64)
    print("\n",lb," 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 17513 samples
Epoch 1/10
17513/17513 [==============================] - 393s 22ms/sample - loss: 0.5152 - acc: 0.7401
Epoch 2/10
17513/17513 [==============================] - 403s 23ms/sample - loss: 0.2658 - acc: 0.9181
Epoch 3/10
17513/17513 [==============================] - 406s 23ms/sample - loss: 0.1381 - acc: 0.9657
Epoch 4/10
17513/17513 [==============================] - 402s 23ms/sample - loss: 0.0697 - acc: 0.9870
Epoch 5/10
17513/17513 [==============================] - 406s 23ms/sample - loss: 0.0401 - acc: 0.9943
Epoch 6/10
17513/17513 [==============================] - 401s 23ms/sample - loss: 0.0235 - acc: 0.9975
Epoch 7/10
17513/17513 [==============================] - 402s 23ms/sample - loss: 0.0141 - acc: 0.9991
Epoch 8/10
17513/17513 [==============================] - 405s 23ms/sample - loss: 0.0083 - acc: 0.9999
Epoch 9/10
17513/17513 [==============================] - 406s 23ms/sample - loss: 0.0065 - acc: 0.9999
Epoch 10/10
1946/1946 [==================